# Data Loading and Preprocessing

In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the paths to your dataset directories
train_dir = "/kaggle/input/deepfake-and-real-images/Dataset/Train"
validation_dir = "/kaggle/input/deepfake-and-real-images/Dataset/Validation"
test_dir = "/kaggle/input/deepfake-and-real-images/Dataset"

# Define image data generators for training, validation, and test sets
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Set batch size
batch_size = 64

# Load and preprocess the training dataset
train_data_v2M = train_datagen.flow_from_directory(
    train_dir,
    target_size=(260, 260),  # Xception requires input size of 299x299 , efficientnet- 260x260
    batch_size=batch_size,
    class_mode='binary'  # Set class_mode according to your dataset
)
train_data_x = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),  # Xception requires input size of 299x299 , efficientnet- 260x260
    batch_size=batch_size,
    class_mode='binary'  # Set class_mode according to your dataset
)

# Load and preprocess the validation dataset
validation_data_v2M = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(260, 260),
    batch_size=batch_size,
    class_mode='binary'
)
validation_data_x = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='binary'
)


# Load and preprocess the test dataset
test_data_v2M = test_datagen.flow_from_directory(
    test_dir,
    target_size=(260, 260),
    batch_size=batch_size,
    class_mode='binary'
)
test_data_x = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='binary'
)

Found 140002 images belonging to 2 classes.
Found 140002 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.
Found 190335 images belonging to 3 classes.
Found 190335 images belonging to 3 classes.


# Xception

In [10]:
import keras
from keras.applications import Xception
from keras.optimizers import Adam
import torch

# Load the pre-trained Xception model without the top layers
base_model = Xception(weights='imagenet', input_shape=(299, 299, 3), include_top=False)

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Add a new output layer for binary classification
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)  # 2 classes for binary classification

# Create the fine-tuned model
model = keras.models.Model(inputs=base_model.input, outputs=outputs)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
with tf.device('/GPU:0'):
    history = model.fit(train_data_x, epochs=10, validation_data=validation_data_x)

# Evaluate the model on the test set
with tf.device('/GPU:0'):
    loss, accuracy = model.evaluate(test_data_x)
    print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 690s 311ms/step - accuracy: 0.7399 - loss: 0.5230 - val_accuracy: 0.7771 - val_loss: 0.4671
Epoch 2/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 686s 313ms/step - accuracy: 0.7814 - loss: 0.4592 - val_accuracy: 0.7709 - val_loss: 0.4736
Epoch 3/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 683s 312ms/step - accuracy: 0.7906 - loss: 0.4416 - val_accuracy: 0.7784 - val_loss: 0.4626
Epoch 4/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 696s 317ms/step - accuracy: 0.7922 - loss: 0.4375 - val_accuracy: 0.7721 - val_loss: 0.4708
Epoch 5/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 695s 317ms/step - accuracy: 0.7951 - loss: 0.4328 - val_accuracy: 0.7846 - val_loss: 0.4560
Epoch 6/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 775s 353ms/step - accuracy: 0.7955 - loss: 0.4301 - val_accuracy: 0.7778 - val_loss: 0.4628
Epoch 7/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 733s 334ms/step - accuracy: 0.7984 - loss: 0.4277 - val_accuracy: 0.7839 - val_loss: 0.4543
Epoch 8/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 686s 313ms/step - ac

# EfficientNetV2M

In [11]:
import tensorflow.keras as keras
from tensorflow.keras.applications import EfficientNetV2M
from tensorflow.keras.optimizers import Adam

# Step 1: Load the EfficientNetV2-M model
model = EfficientNetV2M(weights='imagenet', input_shape=(260, 260, 3), include_top=False)

# Step 2: Freeze the convolutional base
model.trainable = False

# Step 3: Add custom classification head
x = keras.layers.GlobalAveragePooling2D()(model.output)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

# Step 4: Compile the model
model = keras.models.Model(inputs=model.input, outputs=outputs)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Step 5: Train the model
with tf.device('/GPU:0'):
    history = model.fit(train_data_v2M, epochs=10, validation_data=validation_data_v2M)

# Step 6: Evaluate the model
with tf.device('/GPU:0'):
    loss, accuracy = model.evaluate(test_data_v2M)
    print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


Epoch 1/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 972s 388ms/step - accuracy: 0.5064 - loss: 0.6946 - val_accuracy: 0.5021 - val_loss: 0.6924
Epoch 2/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 807s 348ms/step - accuracy: 0.5174 - loss: 0.6923 - val_accuracy: 0.5056 - val_loss: 0.6906
Epoch 3/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 730s 333ms/step - accuracy: 0.5268 - loss: 0.6911 - val_accuracy: 0.5344 - val_loss: 0.6889
Epoch 4/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 722s 329ms/step - accuracy: 0.5339 - loss: 0.6898 - val_accuracy: 0.4981 - val_loss: 0.6975
Epoch 5/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 747s 333ms/step - accuracy: 0.5390 - loss: 0.6882 - val_accuracy: 0.5351 - val_loss: 0.6872
Epoch 7/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 724s 330ms/step - accuracy: 0.5401 - loss: 0.6880 - val_accuracy: 0.5752 - val_loss: 0.6843
Epoch 8/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 720s 328ms/step - accuracy: 0.5452 - loss: 0.6868 - val_accuracy: 0.5506 - val_loss: 0.6854
Epoch 9/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 727s 332ms/step - ac